# Load Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium import plugins
from folium.plugins import HeatMap

plt.rcParams["font.family"] = "Garamond"
plt.rcParams['axes.facecolor'] = "#E8ECEB"
plt.rcParams['axes.facecolor'] = "#E8ECEB"

cols_of_interest = ['Opened', 'Closed', 'Updated', 'Status', 'Status Notes', 'Responsible Agency', 'Category', 'Request Type', 'Request Details', 'Address', 'Supervisor District', 'Neighborhood', 'Latitude', 'Longitude', 'Source', 'Media URL']

data = pd.read_csv('Data/311_cases.csv', usecols=cols_of_interest)

C:\Users\danie\AppData\Local\Temp\ipykernel_14236\4129734305.py:15: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/311_cases.csv', usecols=cols_of_interest)


In [2]:
# convert to datetime
data['Opened'] = pd.to_datetime(data['Opened'])
# create a column called yearmonth that contain the year and month of the opened date
data['yearmonth'] = data['Opened'].apply(lambda x: str(x.year) + '-' + str(x.month) + '-01')

# take dataframe and get a count of every Request Details value summed by month and year
df = data[(data['Category']=="Street and Sidewalk Cleaning")]
# remove rows where latitude or longitude is null or zero
df = df[(df['Latitude']!=0) | (df['Longitude']!=0)]

### Fix Request details types that have different names but are the same
## Garbage and Debris
df['Request Details'] = df['Request Details'].replace(['Other Loose Garbage', 'Other Loose Garbage / Debris', 'Auto_accident_debris',
                                                       'Garbage and Debris - N/A','Garbage and Debris - {{nature_of_request.label}}',
                                                        'Garbage and Debris - {{request_type.label}}'],
                                                        "Loose Garbage and Debris")
## Overflowing city garbage can
df['Request Details'] = df['Request Details'].replace(['City_garbage_can_overflowing', 'Overflowing city receptacle or dumpster',
                                                        'Overflowing_City_Receptacle_or_Dumpster'], "Overflowing City Garbage Can")
## Illegal dumping
df['Request Details'] = df['Request Details'].replace(['Illegal Dumping', 'Illegal dumping', 'Illegal_Dumping'], "Illegal Dumping")
## Furniture, mattresses and refrigerators
df['Request Details'] = df['Request Details'].replace(['Furniture', 'Mattress', 'Refrigerator'], "Furniture, Mattresses and Refrigerators")
## Sidewalk or street cleaning
df['Request Details'] = df['Request Details'].replace(['Sidewalk_Cleaning', 'Sidewalk cleaning', 'Street_Cleaning', 'Street cleaning'], "Sidewalk or Street Cleaning")
## Missed Route Mechanical Sweeping
df['Request Details'] = df['Request Details'].replace(['Missed_Route_Mechanical_Sweeping', 'Missed route - mechanical sweeping'], "Missed Route Mechanical Sweeping")


### Rename some values to be more pretty
## Transit_shelter_platform
df['Request Details'] = df['Request Details'].replace(['Transit_shelter_platform'], "Transit Shelter Platform")
## Christmas tree
df['Request Details'] = df['Request Details'].replace(['Christmas_tree'], "Christmas Tree")
## Event_parade_mess
df['Request Details'] = df['Request Details'].replace(['Event_parade_mess'], "Event Parade Mess")
## Missed_TrashRecycleCompost_Collection
df['Request Details'] = df['Request Details'].replace(['Missed_TrashRecycleCompost_Collection'], "Missed Trash Recycle Compost Collection")
## Medical_waste
df['Request Details'] = df['Request Details'].replace(['Medical_waste'], "Medical Waste")
## GarbageTransit
df['Request Details'] = df['Request Details'].replace(['GarbageTransit'], "Garbage Transit")
## OtherTransit
df['Request Details'] = df['Request Details'].replace(['OtherTransit'], "Other Transit")
## PostAbatementInspection
df['Request Details'] = df['Request Details'].replace(['PostAbatementInspection'], "Post Abatement Inspection")


# print the unique values of Request Details and their counts
df['Request Details'].value_counts()

Loose Garbage and Debris                   810434
Furniture, Mattresses and Refrigerators    540803
Boxed or Bagged Items                      287626
Human or Animal Waste                      257617
Overflowing City Garbage Can               142006
Electronics                                 60893
Glass                                       52819
Needles                                     50998
Liquids - Oil/Paint/Other                   27077
Transit Shelter Platform                    15098
Christmas Tree                              10101
Missed Route Mechanical Sweeping             6277
Paper Spill                                  5260
Tires                                        4800
Hazardous Waste Containers                   4442
Illegal Dumping                              1638
Event Parade Mess                             951
Car Batteries                                 390
Missed Trash Recycle Compost Collection       389
Medical Waste                                 301


In [5]:
df['Year'] = df['Opened'].dt.year

df.head()

,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Latitude,Longitude,Source,Media URL,yearmonth,Year
58,2008-11-10 08:10:22,11/12/2008 03:06:59 PM,11/12/2008 03:06:59 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,"28 13TH ST, SAN FRANCISCO, CA, 94103",6.0,South of Market,37.769745,-122.413742,Phone,NaN,2008-11-01,2008
83,2008-10-27 13:07:52,11/27/2008 02:07:00 PM,11/27/2008 02:07:00 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,"327 LINDEN ST, SAN FRANCISCO, CA, 94102",5.0,Hayes Valley,37.776325,-122.423264,Phone,NaN,2008-10-01,2008
204,2008-10-11 08:45:39,10/15/2008 09:37:00 AM,10/15/2008 09:37:00 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,"Furniture, Mattresses and Refrigerators",Intersection of BURROWS ST and GOETTINGEN ST,9.0,Portola,37.728058,-122.407059,Phone,NaN,2008-10-01,2008
541,2008-11-13 09:03:38,11/13/2008 11:06:56 AM,11/13/2008 11:06:56 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,Intersection of JUNIPERO SERRA BLVD and SLOAT ...,7.0,Merced Manor,37.734707,-122.471573,Phone,NaN,2008-11-01,2008
584,2008-10-13 12:46:42,10/14/2008 06:06:54 AM,10/14/2008 06:06:54 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,"Furniture, Mattresses and Refrigerators",Intersection of LEAVENWORTH ST and SUTTER ST,3.0,Lower Nob Hill,37.788387,-122.415184,Phone,NaN,2008-10-01,2008


In [12]:
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump(df, f)

# Loading Shortcut

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium import plugins
from folium.plugins import HeatMap

plt.rcParams["font.family"] = "Garamond"
plt.rcParams['axes.facecolor'] = "#E8ECEB"
plt.rcParams['axes.facecolor'] = "#E8ECEB"

with open('data.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
import plotly.graph_objs as go
import pandas as pd

yearly_counts = df.groupby('Year').size()
yearly_counts = yearly_counts.drop(2023)

hover_text = [f"<b>{val/1000:.1f}k</b> <extra></extra>" if val >= 1000 else str(val) for val in yearly_counts.values]

fig = go.Figure(go.Bar(
    x=yearly_counts.index,
    y=yearly_counts.values,
    hovertemplate=hover_text,
    marker_color='limegreen',
    marker_line_color='black',
    marker_line_width=1,
    hoverlabel=dict(bgcolor="gold")
))

# Set x-axis ticks to show all years
fig.update_layout(xaxis=dict(dtick=1))

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Number of Street & Sidewalk Requests",
    title="<b>San Francisco Street & Sidewalk Cleaning Requests</b>"
)

fig.write_html('plot.html')
fig.show()
